# <center> Oriol Masias Vergés </center>
# <center> Analyzing Bank Marketing Data with Python and Sqlite3 </center>

In the field of **data analytics**, understanding and extracting insights from datasets play a vital role in making **informed decisions** and gaining valuable business intelligence. To sharpen my data analytics skills, I embark on a project centered around the **Bank Marketing Dataset**. This dataset contains information about a bank's marketing campaigns and customer attributes, offering a rich resource for exploration and analysis. Each entry in the dataset represents an **individual customer** and their interaction with the bank's marketing campaigns.

## Variables:

- **age:** Age of the client.
- **job:** Type of job of the client.
- **marital:** Marital status of the client.
- **education:** Level of education of the client.
- **default:** Whether the client has a credit in default or not.
- **balance:** Balance of the client's account.
- **housing:** Whether the client has a housing loan or not.
- **loan:** Whether the client has a personal loan or not.
- **contact:** Contact communication type with the client.
- **day:** Day of the month when the client was last contacted.
- **month:** Month of the year when the client was last contacted.
- **duration:** Duration of the last contact with the client in seconds.
- **campaign:** Number of contacts performed during this campaign for the client.
- **pdays:** Number of days that passed after the client was last contacted from a previous campaign (A value of 999 indicates that the client was not previously contacted).
- **previous:** Number of contacts performed before this campaign and for this client.
- **poutcome:** Outcome of the previous marketing campaign.
- **deposit:** Whether the client subscribed to a term deposit or not.

In [1]:
# Import libreries
import sqlite3
from pathlib import Path
import pandas as pd

In [2]:
# Create empty database
Path("bank.db").touch()

In [3]:
# Connect to database
conn = sqlite3.connect("bank.db")
c = conn.cursor()

In [4]:
# Drop table if exists
c.execute("""DROP TABLE IF EXISTS bank;""")

# Create a table
c.execute(
    """CREATE TABLE bank (
    age int, job text, 
    marital text, education text, 
    default_e text, balance int, 
    housing text, loan text, 
    contact text, day int, month text, 
    duration int, campaign int, 
    pdays text, previous int, 
    poutcome text, deposit text
    );"""
)

In [5]:
# Open csv file
bank = pd.read_csv("bank.csv")

# Add table to database
bank.to_sql("bank", conn, if_exists="append", index=False)

11162

In [6]:
# Create a function to print results
def print_rows(cursor):
    column_names = [
        "Age", "Job", "Marital", "Education", "Default", "Balance", "Housing",
        "Loan", "Contact", "Day", "Month", "Duration", "Campaign", "Pdays",
        "Previous", "Poutcome", "Deposit"
    ]
    
    for row in rows:
        for i, column_value in enumerate(row):
            print(f"{column_names[i]}:", column_value)
        print("---------------")

# Display 10 rows of the table
rows = c.execute("""SELECT * FROM bank;""").fetchmany(5)
print_rows(rows)

Age: 59
Job: admin.
Marital: married
Education: secondary
Default: no
Balance: 2343
Housing: yes
Loan: no
Contact: unknown
Day: 5
Month: may
Duration: 1042
Campaign: 1
Pdays: -1
Previous: 0
Poutcome: unknown
Deposit: yes
---------------
Age: 56
Job: admin.
Marital: married
Education: secondary
Default: no
Balance: 45
Housing: no
Loan: no
Contact: unknown
Day: 5
Month: may
Duration: 1467
Campaign: 1
Pdays: -1
Previous: 0
Poutcome: unknown
Deposit: yes
---------------
Age: 41
Job: technician
Marital: married
Education: secondary
Default: no
Balance: 1270
Housing: yes
Loan: no
Contact: unknown
Day: 5
Month: may
Duration: 1389
Campaign: 1
Pdays: -1
Previous: 0
Poutcome: unknown
Deposit: yes
---------------
Age: 55
Job: services
Marital: married
Education: secondary
Default: no
Balance: 2476
Housing: yes
Loan: no
Contact: unknown
Day: 5
Month: may
Duration: 579
Campaign: 1
Pdays: -1
Previous: 0
Poutcome: unknown
Deposit: yes
---------------
Age: 54
Job: admin.
Marital: married
Education: te

In [7]:
# Which professions are the most popular among customers over 45 years old?
c.execute("""SELECT job, COUNT(job) 
                FROM bank
                WHERE age > 45
                GROUP BY job
                ORDER BY COUNT(job) DESC;
                """).fetchall()

[('retired', 764),
 ('management', 757),
 ('blue-collar', 537),
 ('technician', 456),
 ('admin.', 361),
 ('services', 206),
 ('housemaid', 157),
 ('entrepreneur', 125),
 ('unemployed', 118),
 ('self-employed', 113),
 ('unknown', 37),
 ('student', 2)]

In [8]:
# For how many people with loans did the marketing campaign succeed?
c.execute("""SELECT COUNT(*) 
                FROM bank
                WHERE loan == "yes" AND Poutcome = "success";""").fetchone()[0]

51

In [9]:
# How many unique job types are there in the dataset?
c.execute("""SELECT COUNT(DISTINCT job) FROM bank;""").fetchone()[0]

12

In [10]:
# What is the average age of clients who suscribed to a term deposit?
c.execute("""SELECT AVG(age) 
                FROM bank
                WHERE deposit = "yes";""").fetchone()[0]

41.670069956513515

In [11]:
# Which month had the highest number of marketing campaign contacts?
c.execute("""SELECT month, SUM(campaign) 
                FROM bank
                GROUP BY month
                ORDER BY SUM(campaign) DESC
                LIMIT 1;""").fetchall()

[('may', 6575)]

In [12]:
# How many clients have the highest balance in their respective job categories?
c.execute("""SELECT job, COUNT(*) AS client_count 
                FROM bank 
                WHERE balance = (SELECT MAX(balance)
                                    FROM bank b2
                                    WHERE b2.job = bank.job)
                GROUP BY job;""").fetchall()

[('admin.', 1),
 ('blue-collar', 1),
 ('entrepreneur', 1),
 ('housemaid', 1),
 ('management', 1),
 ('retired', 2),
 ('self-employed', 2),
 ('services', 1),
 ('student', 2),
 ('technician', 1),
 ('unemployed', 1),
 ('unknown', 1)]

In [13]:
# What is the maximum number of contacts made in a single day during the marketing campaign?
c.execute("""SELECT day, SUM(campaign)
                FROM bank
                GROUP BY day
                ORDER BY SUM(campaign) DESC
                LIMIT 1""").fetchall()[0]

(20, 1617)

In [14]:
# Determine the percentage of clients who have subscribed to a term deposit within each marital group.
c.execute("""SELECT marital, (COUNT(CASE WHEN deposit = 'yes' THEN 1 END) * 100.0 / COUNT(*)) AS percentage
                FROM bank
                GROUP BY marital;""").fetchall()

[('divorced', 48.10518174787316),
 ('married', 43.37899543378995),
 ('single', 54.34906196702672)]

In [15]:
# Find the job category that has the highest average balance among clients who have not subscribed to a term deposit.
c.execute("""SELECT job, ROUND(AVG(balance),2)
                FROM bank
                WHERE deposit = "no"
                GROUP BY job
                ORDER BY AVG(balance) DESC;""").fetchall()

[('retired', 1878.84),
 ('unknown', 1674.0),
 ('student', 1536.38),
 ('entrepreneur', 1503.72),
 ('self-employed', 1448.11),
 ('management', 1436.74),
 ('technician', 1315.44),
 ('unemployed', 1289.26),
 ('blue-collar', 1162.97),
 ('housemaid', 1083.87),
 ('services', 1060.41),
 ('admin.', 979.17)]

In [16]:
# Identify the martial satatus that has the highest percentage of clients with a housing loan and a term deposit.
c.execute("""SELECT marital, ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*)
                                                    FROM bank 
                                                    WHERE housing = 'yes' AND deposit = 'yes'),2) AS percentage
                FROM bank
                WHERE housing = 'yes' AND deposit = 'yes'
                GROUP BY marital
                ORDER BY percentage DESC""").fetchall()

[('married', 52.14), ('single', 34.99), ('divorced', 12.87)]

## Conclusions
- The most popular professions among customers over 45 years old are retired (764), management (757) and blue-collar (537).
- The last marketing campaign succeed for 51 people with loans.
- There are 12 unique job types in the dataset.
- The average age of clients who suscribed to a term deposit is 41 years.
- The month with the highest number of marketing campaign contacts is may (6575).
- The maximum number of contacts made in a single day during the marketing campaign are 1617.
- The percentage of clients who have subscribed to a term deposit within each martial group are:
    - Divorced: 48.11%
    - Married: 43.39%
    - Single: 54.35%
- The job category that has the highest average balance among clients who have not subscribed to a term deposit is retired (1878.84).
- The martial status that has the highest percentage of clients with a housing loan and a term deposit is married (52.14%).

In [17]:
# Close the database
c.close()